### **Generacion clasificada de la informacion**

En el siguiente documento, se realiza la exploracion de la informacion, esto con el fin de poder segmentar el dataset correctamente para reducir el procesamiento. 

Para conocimiento, se tomaran unicamente 6 clases de las 800 (aproximadamente) para el entrenamieto, siendo asi, de un total de mas 700,000 imagenes para tomar al menos 100 imagenes por clase (considerando aumentar la data dependiendo del tiempo del entrnamiento).

Las clases tienen la cualidad de estar divididas en al menos 800 clases, que desde el punto astronomico se consideran al menos 3 clases y teniendo el resto como subclases determinadas de las caracteristicas propias de cada galaxia.

In [1]:
######################################################################################################################################################
# Importación de librerías, modelos y rutas                                                                                                          #
######################################################################################################################################################

import pandas as pd
import cv2
import json
import os

# Ruta de imágenes
image_path="./data-set/"
# Ruta de los CSV
information_path="./information/"

# Carga el CSV de características
df_characteristics=pd.read_csv(f"{information_path}characteristics.csv")
# Carga el CSV de nombres
df_name=pd.read_csv(f"{information_path}names.csv")

#### **Extraccion de clases unicas**

Primero que nada devemos conocer la naturaleza de las clases, esto para saber como fragmentar de manera precisa las clases y poder extraer las mas caracteristicas. Para el sigueinte proceso, primero exploramos el total de clases, extrallendo las clases unicas, para despues seleccionar una parte de ellas como dataset de entrenamiento y generar un txt con las imagenes de las clases, de las cuales no seleccioanmos para el entrenamiento, siendo estas las dedicadas para las pruebas.

Ahora que sabemos que el dataset contiene por lo menos 800 clases, podemos decirdir extraer un total de 6 clases, dado que el dataset tambien es de gran tamaño

In [2]:
# Extraemos los clases unicas
unique_classes=df_characteristics["gz2_class"].unique()

# Grabamos las clases en un txt general
with open("clases.txt", "a") as file:
    for unique_class in unique_classes:
        file.write(f"{unique_class}\n")

# Hacemos un plot del total de clases
print(len(unique_classes))

# Para el entrenamiento de la rnn solousarenmos las dos tipos de espiral normales, dos de tipo barridas y dos de tipo elipticas
classes_en=["Sb+t", "Sc+t", "SBb2m", "SBc3m", "Er", "Ec"]

# Posteriormente extraemos al menos 100 ID de imagenes que cumplan con esas clases
class_information={}
for unique_class in classes_en:
    unique_images=df_characteristics[df_characteristics["gz2_class"]==unique_class]
    unique_images_class={}
    count=0
    for sidx, srow in unique_images.iterrows():
        if count==5000:
            break
        # Extraemos el nombre de la imagen
        image_path=df_name[df_name["objid"]==srow["dr7objid"]]
        image_path=image_path["asset_id"].iloc[0]
        unique_images_class.update({f"{image_path}": srow["dr7objid"]})
        count+=1
    class_information[unique_class]=unique_images_class

818


In [3]:
with open("class_images.json", "w") as file:
    json.dump(class_information, file, indent=4)
print(class_information)

{'Sb+t': {'193641': 588009368545984617, '87614': 587735742617616406, '93683': 587736584436777018, '97356': 587736782539915626, '140392': 587739810486616170, '6541': 587725469052502129, '83693': 587735661552599116, '47604': 587731873923530987, '111900': 587738568177221727, '159401': 587741727112101895, '166903': 587742061061996578, '71922': 587733609085337761, '17218': 587726031701540990, '83642': 587735661549256781, '154114': 587741600953598078, '161326': 587741828580245736, '218669': 588017704542994539, '165115': 587742014903418934, '85279': 587735666380243001, '90682': 587736541473931384, '224297': 588017947741323392, '133955': 587739647284084849, '22401': 587727943493550328, '52299': 587732136996765809, '161647': 587741829656084695, '212017': 588017604684677270, '155468': 587741603103047720, '219500': 588017712045621269, '50182': 587732050555306142, '127459': 587739407312355333, '229436': 588017979975532720, '130187': 587739505005887673, '19300': 587726033316544779, '236100': 588023

#### **Generacion de CSV final**

Una vez ya tenemos las imagenes unicas por cada clase podemos pasar a generar los vectores para el entrenamiento de la Red Neuronal, esto implñica hacer un proprocesamiento a las imagenes de manera global y standarizada con el fin de poder extraer los vectores mas estandarizados, esto para que la red neuronal no tenga problemas con la clasificacion posterior de las nuevas imagenes de entrada

El punto de poder vectorizar la informacion es para poder ser procesada por una red neuronal ya que etsas trabajan con valores de entr un uno a un cero, es decir, un rango normalizado, la red neuronal interpreta y modifca los pesos, para extraer un patron y esto nos permite definir que tan probable de que pertenezca a una clase es

In [4]:
# Definimos la funcion que se encargara de vectorizar una imagen
def vectorize_image(image_path):
    # Llevamos a gris la imagen
    img=cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Eliminamos ruido con un filtro gaussiano
    img_suavizad=cv2.GaussianBlur(img, (21, 21), 0)
    # Redimensionar la imagen
    img_resized=cv2.resize(img_suavizad, (64, 64))
    # Aplanar la imagen en un vector unidimensional
    img_vector=img_resized.flatten()
    # Normalizar los valores del vector a un rango [0, 1]
    img_vector=img_vector / 255.0
    return img_vector

# Prodemos a acceder a todas las imagenes y generar sus vectores apra escribirlos en un csv
data=[]
for key_class in class_information:
    for key_image, value in class_information[key_class].items():
        # Verificamos existencia de imagen
        if not os.path.exists(f"./data-set/{key_image}.jpg"):
            print(f"Imagen {key_image} no encontrada")
        else:
            image_vector=vectorize_image(f"./data-set/{key_image}.jpg")
            data.append({
                "image_name": key_image,
                "image_vector": image_vector.tolist(),
                "galaxy_class": key_class
            })

Imagen 26603 no encontrada


In [5]:
# Guardamos el CSV
output_csv="dataset-vectors.csv" 
df_vectors=pd.DataFrame(data)
df_vectors.to_csv(output_csv, index=False)

print(f"Archivo CSV generado: {output_csv}")

Archivo CSV generado: dataset-vectors.csv
